In [1]:
import pandas as pd
import re
from datetime import datetime
import nltk
nltk.download(['punkt', 'stopwords', 'wordnet', 'omw-1.4', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

pd.set_option('display.max_columns', None)

[nltk_data] Downloading package punkt to /home/jakeli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jakeli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jakeli/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jakeli/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jakeli/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Import Data

In [2]:
df = pd.read_csv('data/anime_data.csv')
df.head()

,id,title,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,num_episodes,source,average_episode_duration,rating,main_picture.medium,main_picture.large,alternative_titles.synonyms,alternative_titles.en,alternative_titles.ja,start_season.year,start_season.season,broadcast.day_of_the_week,broadcast.start_time,genres,studios
0,5114,Fullmetal Alchemist: Brotherhood,2009-04-05,2010-07-04,After a horrific alchemy experiment goes wrong...,9.10,1,3,3162045,2011391,white,2008-08-21T03:35:22+00:00,2023-04-02T18:07:03+00:00,tv,finished_airing,64,manga,1460,r,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,['Hagane no Renkinjutsushi: Fullmetal Alchemis...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,2009.0,spring,sunday,17:00,"Action, Adventure, Drama, Fantasy, Military, S...",Bones
1,41467,Bleach: Sennen Kessen-hen,2022-10-11,2022-12-27,Substitute Soul Reaper Ichigo Kurosaki spends ...,9.07,2,474,435441,207104,white,2020-03-18T09:10:15+00:00,2023-04-02T18:07:55+00:00,tv,finished_airing,13,manga,1471,r,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,['Bleach: Thousand-Year Blood War Arc'],Bleach: Thousand-Year Blood War,BLEACH 千年血戦篇,2022.0,fall,tuesday,00:00,"Action, Adventure, Fantasy, Shounen",Pierrot
2,9253,Steins;Gate,2011-04-06,2011-09-14,Eccentric scientist Rintarou Okabe has a never...,9.07,3,13,2429238,1330574,white,2010-07-26T09:23:40+00:00,2023-04-02T18:08:42+00:00,tv,finished_airing,24,visual_novel,1460,pg_13,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[],Steins;Gate,STEINS;GATE,2011.0,spring,wednesday,02:05,"Drama, Psychological, Sci-Fi, Suspense, Time, ...","White, Fox"
3,38524,Shingeki no Kyojin Season 3 Part 2,2019-04-29,2019-07-01,Seeking to restore humanity's diminishing hope...,9.06,4,24,2089588,1462437,white,2018-10-14T18:13:45+00:00,2023-04-02T18:10:19+00:00,tv,finished_airing,10,manga,1435,r,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[],Attack on Titan Season 3 Part 2,進撃の巨人 Season3 Part.2,2019.0,spring,monday,00:10,"Action, Drama, Gore, Military, Shounen, Survival","Wit, Studio"
4,51535,Shingeki no Kyojin: The Final Season - Kankets...,2023-03-04,2023,In the wake of Eren Yeager's cataclysmic actio...,9.06,5,497,421519,147335,white,2022-04-03T15:34:50+00:00,2023-05-21T09:02:22+00:00,special,currently_airing,2,manga,3690,r,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,['Shingeki no Kyojin: The Final Season Part 3'...,Attack on Titan: The Final Season - Final Chap...,進撃の巨人 The Final Season完結編,2023.0,winter,NaN,NaN,"Action, Drama, Gore, Military, Shounen, Surviv...",MAPPA


# EDA + Data Cleaning

In [3]:
df.shape

(10000, 30)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           10000 non-null  int64  
 1   title                        10000 non-null  object 
 2   start_date                   9995 non-null   object 
 3   end_date                     9904 non-null   object 
 4   synopsis                     9847 non-null   object 
 5   mean                         10000 non-null  float64
 6   rank                         10000 non-null  int64  
 7   popularity                   10000 non-null  int64  
 8   num_list_users               10000 non-null  int64  
 9   num_scoring_users            10000 non-null  int64  
 10  nsfw                         10000 non-null  object 
 11  created_at                   10000 non-null  object 
 12  updated_at                   10000 non-null  object 
 13  media_type       

In [5]:
# convert columns to the best possible dtypes using convert_dtypes function
df = df.convert_dtypes()
df.dtypes

id                                      Int64
title                          string[python]
start_date                     string[python]
end_date                       string[python]
synopsis                       string[python]
mean                                  Float64
rank                                    Int64
popularity                              Int64
num_list_users                          Int64
num_scoring_users                       Int64
nsfw                           string[python]
created_at                     string[python]
updated_at                     string[python]
media_type                     string[python]
status                         string[python]
num_episodes                            Int64
source                         string[python]
average_episode_duration                Int64
rating                         string[python]
main_picture.medium            string[python]
main_picture.large             string[python]
alternative_titles.synonyms    str

In [6]:
df.head()

,id,title,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,num_episodes,source,average_episode_duration,rating,main_picture.medium,main_picture.large,alternative_titles.synonyms,alternative_titles.en,alternative_titles.ja,start_season.year,start_season.season,broadcast.day_of_the_week,broadcast.start_time,genres,studios
0,5114,Fullmetal Alchemist: Brotherhood,2009-04-05,2010-07-04,After a horrific alchemy experiment goes wrong...,9.1,1,3,3162045,2011391,white,2008-08-21T03:35:22+00:00,2023-04-02T18:07:03+00:00,tv,finished_airing,64,manga,1460,r,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,['Hagane no Renkinjutsushi: Fullmetal Alchemis...,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,2009,spring,sunday,17:00,"Action, Adventure, Drama, Fantasy, Military, S...",Bones
1,41467,Bleach: Sennen Kessen-hen,2022-10-11,2022-12-27,Substitute Soul Reaper Ichigo Kurosaki spends ...,9.07,2,474,435441,207104,white,2020-03-18T09:10:15+00:00,2023-04-02T18:07:55+00:00,tv,finished_airing,13,manga,1471,r,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,['Bleach: Thousand-Year Blood War Arc'],Bleach: Thousand-Year Blood War,BLEACH 千年血戦篇,2022,fall,tuesday,00:00,"Action, Adventure, Fantasy, Shounen",Pierrot
2,9253,Steins;Gate,2011-04-06,2011-09-14,Eccentric scientist Rintarou Okabe has a never...,9.07,3,13,2429238,1330574,white,2010-07-26T09:23:40+00:00,2023-04-02T18:08:42+00:00,tv,finished_airing,24,visual_novel,1460,pg_13,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[],Steins;Gate,STEINS;GATE,2011,spring,wednesday,02:05,"Drama, Psychological, Sci-Fi, Suspense, Time, ...","White, Fox"
3,38524,Shingeki no Kyojin Season 3 Part 2,2019-04-29,2019-07-01,Seeking to restore humanity's diminishing hope...,9.06,4,24,2089588,1462437,white,2018-10-14T18:13:45+00:00,2023-04-02T18:10:19+00:00,tv,finished_airing,10,manga,1435,r,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,[],Attack on Titan Season 3 Part 2,進撃の巨人 Season3 Part.2,2019,spring,monday,00:10,"Action, Drama, Gore, Military, Shounen, Survival","Wit, Studio"
4,51535,Shingeki no Kyojin: The Final Season - Kankets...,2023-03-04,2023,In the wake of Eren Yeager's cataclysmic actio...,9.06,5,497,421519,147335,white,2022-04-03T15:34:50+00:00,2023-05-21T09:02:22+00:00,special,currently_airing,2,manga,3690,r,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,['Shingeki no Kyojin: The Final Season Part 3'...,Attack on Titan: The Final Season - Final Chap...,進撃の巨人 The Final Season完結編,2023,winter,<NA>,<NA>,"Action, Drama, Gore, Military, Shounen, Surviv...",MAPPA


In [7]:
# remove brackets and quotation marks from titles
df['title'] = df['title'].apply(lambda x: re.sub('[\[\]\'"]', '', x))
df['alternative_titles.synonyms'] = df['alternative_titles.synonyms'].apply(lambda x: re.sub('[\[\]\'"]', '', x))
df['alternative_titles.en'] = df['alternative_titles.en'].apply(lambda x: re.sub('[\[\]\'"]', '', str(x)))
df['alternative_titles.ja'] = df['alternative_titles.ja'].apply(lambda x: re.sub('[\[\]\'"]', '', str(x)))

df.head(3)

,id,title,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,num_episodes,source,average_episode_duration,rating,main_picture.medium,main_picture.large,alternative_titles.synonyms,alternative_titles.en,alternative_titles.ja,start_season.year,start_season.season,broadcast.day_of_the_week,broadcast.start_time,genres,studios
0,5114,Fullmetal Alchemist: Brotherhood,2009-04-05,2010-07-04,After a horrific alchemy experiment goes wrong...,9.1,1,3,3162045,2011391,white,2008-08-21T03:35:22+00:00,2023-04-02T18:07:03+00:00,tv,finished_airing,64,manga,1460,r,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,"Hagane no Renkinjutsushi: Fullmetal Alchemist,...",Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,2009,spring,sunday,17:00,"Action, Adventure, Drama, Fantasy, Military, S...",Bones
1,41467,Bleach: Sennen Kessen-hen,2022-10-11,2022-12-27,Substitute Soul Reaper Ichigo Kurosaki spends ...,9.07,2,474,435441,207104,white,2020-03-18T09:10:15+00:00,2023-04-02T18:07:55+00:00,tv,finished_airing,13,manga,1471,r,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,Bleach: Thousand-Year Blood War Arc,Bleach: Thousand-Year Blood War,BLEACH 千年血戦篇,2022,fall,tuesday,00:00,"Action, Adventure, Fantasy, Shounen",Pierrot
2,9253,Steins;Gate,2011-04-06,2011-09-14,Eccentric scientist Rintarou Okabe has a never...,9.07,3,13,2429238,1330574,white,2010-07-26T09:23:40+00:00,2023-04-02T18:08:42+00:00,tv,finished_airing,24,visual_novel,1460,pg_13,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,,Steins;Gate,STEINS;GATE,2011,spring,wednesday,02:05,"Drama, Psychological, Sci-Fi, Suspense, Time, ...","White, Fox"


In [8]:
# check duplicated rows and IDs
print('Duplicates:', df.duplicated().sum())

print('ID duplicates:', df.duplicated(subset='id').sum())

Duplicates: 0
ID duplicates: 0


In [9]:
# check nulls
df.isnull().sum()

id                                0
title                             0
start_date                        5
end_date                         96
synopsis                        153
mean                              0
rank                              0
popularity                        0
num_list_users                    0
num_scoring_users                 0
nsfw                              0
created_at                        0
updated_at                        0
media_type                        0
status                            0
num_episodes                      0
source                         1051
average_episode_duration          0
rating                           71
main_picture.medium               1
main_picture.large                1
alternative_titles.synonyms       0
alternative_titles.en             0
alternative_titles.ja             0
start_season.year               172
start_season.season             172
broadcast.day_of_the_week      7186
broadcast.start_time        

In [10]:
print('(Rows, Columns) before dropping:', df.shape)

# drop rows where the synopsis/plot is null
df.dropna(subset=['synopsis'], inplace=True)
# drop the broadcast.day_of_the_week and broadcast.start_time columns
df.drop(['broadcast.day_of_the_week', 'broadcast.start_time'], axis=1, inplace=True)

print('(Rows, Columns) after dropping:', df.shape)

(Rows, Columns) before dropping: (10000, 30)
(Rows, Columns) after dropping: (9847, 28)


In [11]:
# check the range of year
print('Minimum year:', df['start_season.year'].min())
print('Maximum year:', df['start_season.year'].max())

Minimum year: 1917
Maximum year: 2023


In [12]:
# filter out anime that aired before 2005
df = df[df['start_season.year'] >= 2005]
df.shape

(6836, 28)

In [13]:
# check synopsis
print(df['synopsis'][0], '\n')
print(df['synopsis'][1])

After a horrific alchemy experiment goes wrong in the Elric household, brothers Edward and Alphonse are left in a catastrophic new reality. Ignoring the alchemical principle banning human transmutation, the boys attempted to bring their recently deceased mother back to life. Instead, they suffered brutal personal loss: Alphonse's body disintegrated while Edward lost a leg and then sacrificed an arm to keep Alphonse's soul in the physical realm by binding it to a hulking suit of armor.

The brothers are rescued by their neighbor Pinako Rockbell and her granddaughter Winry. Known as a bio-mechanical engineering prodigy, Winry creates prosthetic limbs for Edward by utilizing "automail," a tough, versatile metal used in robots and combat armor. After years of training, the Elric brothers set off on a quest to restore their bodies by locating the Philosopher's Stone—a powerful gem that allows an alchemist to defy the traditional laws of Equivalent Exchange.

As Edward becomes an infamous al

In [14]:
# get rid of the string '[Written by MAL Rewrite]' at the end of synopsis
df['synopsis'] = df['synopsis'].apply(lambda x: str(x).replace('[Written by MAL Rewrite]', ''))

print(df['synopsis'][0], '\n')
print(df['synopsis'][1])

After a horrific alchemy experiment goes wrong in the Elric household, brothers Edward and Alphonse are left in a catastrophic new reality. Ignoring the alchemical principle banning human transmutation, the boys attempted to bring their recently deceased mother back to life. Instead, they suffered brutal personal loss: Alphonse's body disintegrated while Edward lost a leg and then sacrificed an arm to keep Alphonse's soul in the physical realm by binding it to a hulking suit of armor.

The brothers are rescued by their neighbor Pinako Rockbell and her granddaughter Winry. Known as a bio-mechanical engineering prodigy, Winry creates prosthetic limbs for Edward by utilizing "automail," a tough, versatile metal used in robots and combat armor. After years of training, the Elric brothers set off on a quest to restore their bodies by locating the Philosopher's Stone—a powerful gem that allows an alchemist to defy the traditional laws of Equivalent Exchange.

As Edward becomes an infamous al

In [15]:
# check index
df.index

Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       9987, 9988, 9989, 9990, 9992, 9993, 9995, 9996, 9997, 9999],
      dtype='int64', length=6836)

In [16]:
# fix index
df.index = pd.RangeIndex(start=0, stop=len(df), step=1)
df.index

RangeIndex(start=0, stop=6836, step=1)

In [17]:
# print columns
df.columns.values

array(['id', 'title', 'start_date', 'end_date', 'synopsis', 'mean',
       'rank', 'popularity', 'num_list_users', 'num_scoring_users',
       'nsfw', 'created_at', 'updated_at', 'media_type', 'status',
       'num_episodes', 'source', 'average_episode_duration', 'rating',
       'main_picture.medium', 'main_picture.large',
       'alternative_titles.synonyms', 'alternative_titles.en',
       'alternative_titles.ja', 'start_season.year',
       'start_season.season', 'genres', 'studios'], dtype=object)

In [18]:
# rename columns
df = df.rename(columns={'main_picture.medium':'medium_picture_url', 'main_picture.large':'large_picture_url',
                        'alternative_titles.synonyms':'alternative_title(s)','alternative_titles.en':'english_title', 
                        'alternative_titles.ja':'japanese_title', 'start_season.year':'year', 'start_season.season':'season'})

df.head(3)

,id,title,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,num_episodes,source,average_episode_duration,rating,medium_picture_url,large_picture_url,alternative_title(s),english_title,japanese_title,year,season,genres,studios
0,5114,Fullmetal Alchemist: Brotherhood,2009-04-05,2010-07-04,After a horrific alchemy experiment goes wrong...,9.1,1,3,3162045,2011391,white,2008-08-21T03:35:22+00:00,2023-04-02T18:07:03+00:00,tv,finished_airing,64,manga,1460,r,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,"Hagane no Renkinjutsushi: Fullmetal Alchemist,...",Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,2009,spring,"Action, Adventure, Drama, Fantasy, Military, S...",Bones
1,41467,Bleach: Sennen Kessen-hen,2022-10-11,2022-12-27,Substitute Soul Reaper Ichigo Kurosaki spends ...,9.07,2,474,435441,207104,white,2020-03-18T09:10:15+00:00,2023-04-02T18:07:55+00:00,tv,finished_airing,13,manga,1471,r,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,Bleach: Thousand-Year Blood War Arc,Bleach: Thousand-Year Blood War,BLEACH 千年血戦篇,2022,fall,"Action, Adventure, Fantasy, Shounen",Pierrot
2,9253,Steins;Gate,2011-04-06,2011-09-14,Eccentric scientist Rintarou Okabe has a never...,9.07,3,13,2429238,1330574,white,2010-07-26T09:23:40+00:00,2023-04-02T18:08:42+00:00,tv,finished_airing,24,visual_novel,1460,pg_13,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,,Steins;Gate,STEINS;GATE,2011,spring,"Drama, Psychological, Sci-Fi, Suspense, Time, ...","White, Fox"


# Text Preprocessing

In [19]:
df.columns.values

array(['id', 'title', 'start_date', 'end_date', 'synopsis', 'mean',
       'rank', 'popularity', 'num_list_users', 'num_scoring_users',
       'nsfw', 'created_at', 'updated_at', 'media_type', 'status',
       'num_episodes', 'source', 'average_episode_duration', 'rating',
       'medium_picture_url', 'large_picture_url', 'alternative_title(s)',
       'english_title', 'japanese_title', 'year', 'season', 'genres',
       'studios'], dtype=object)

In [20]:
# select and combine features that I think it's going to be useful for the recommendation engine
features = ['synopsis', 'genres']
df['summary'] = df[features].apply(lambda x: ' '.join(x.astype(str)), axis=1)

print(df['summary'][0])

After a horrific alchemy experiment goes wrong in the Elric household, brothers Edward and Alphonse are left in a catastrophic new reality. Ignoring the alchemical principle banning human transmutation, the boys attempted to bring their recently deceased mother back to life. Instead, they suffered brutal personal loss: Alphonse's body disintegrated while Edward lost a leg and then sacrificed an arm to keep Alphonse's soul in the physical realm by binding it to a hulking suit of armor.

The brothers are rescued by their neighbor Pinako Rockbell and her granddaughter Winry. Known as a bio-mechanical engineering prodigy, Winry creates prosthetic limbs for Edward by utilizing "automail," a tough, versatile metal used in robots and combat armor. After years of training, the Elric brothers set off on a quest to restore their bodies by locating the Philosopher's Stone—a powerful gem that allows an alchemist to defy the traditional laws of Equivalent Exchange.

As Edward becomes an infamous al

In [21]:
# apply lowercase and tokenize
df['tokens'] = df['summary'].apply(str.lower).apply(word_tokenize)

print(df['tokens'][0])

['after', 'a', 'horrific', 'alchemy', 'experiment', 'goes', 'wrong', 'in', 'the', 'elric', 'household', ',', 'brothers', 'edward', 'and', 'alphonse', 'are', 'left', 'in', 'a', 'catastrophic', 'new', 'reality', '.', 'ignoring', 'the', 'alchemical', 'principle', 'banning', 'human', 'transmutation', ',', 'the', 'boys', 'attempted', 'to', 'bring', 'their', 'recently', 'deceased', 'mother', 'back', 'to', 'life', '.', 'instead', ',', 'they', 'suffered', 'brutal', 'personal', 'loss', ':', 'alphonse', "'s", 'body', 'disintegrated', 'while', 'edward', 'lost', 'a', 'leg', 'and', 'then', 'sacrificed', 'an', 'arm', 'to', 'keep', 'alphonse', "'s", 'soul', 'in', 'the', 'physical', 'realm', 'by', 'binding', 'it', 'to', 'a', 'hulking', 'suit', 'of', 'armor', '.', 'the', 'brothers', 'are', 'rescued', 'by', 'their', 'neighbor', 'pinako', 'rockbell', 'and', 'her', 'granddaughter', 'winry', '.', 'known', 'as', 'a', 'bio-mechanical', 'engineering', 'prodigy', ',', 'winry', 'creates', 'prosthetic', 'limbs',

In [22]:
# remove stop words and keep letters only
stop_words = stopwords.words('english')
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words and word.isalpha()])

print(df['tokens'][0])

['horrific', 'alchemy', 'experiment', 'goes', 'wrong', 'elric', 'household', 'brothers', 'edward', 'alphonse', 'left', 'catastrophic', 'new', 'reality', 'ignoring', 'alchemical', 'principle', 'banning', 'human', 'transmutation', 'boys', 'attempted', 'bring', 'recently', 'deceased', 'mother', 'back', 'life', 'instead', 'suffered', 'brutal', 'personal', 'loss', 'alphonse', 'body', 'disintegrated', 'edward', 'lost', 'leg', 'sacrificed', 'arm', 'keep', 'alphonse', 'soul', 'physical', 'realm', 'binding', 'hulking', 'suit', 'armor', 'brothers', 'rescued', 'neighbor', 'pinako', 'rockbell', 'granddaughter', 'winry', 'known', 'engineering', 'prodigy', 'winry', 'creates', 'prosthetic', 'limbs', 'edward', 'utilizing', 'automail', 'tough', 'versatile', 'metal', 'used', 'robots', 'combat', 'armor', 'years', 'training', 'elric', 'brothers', 'set', 'quest', 'restore', 'bodies', 'locating', 'philosopher', 'powerful', 'gem', 'allows', 'alchemist', 'defy', 'traditional', 'laws', 'equivalent', 'exchange'

In [23]:
#  apply tag to each token to prepare for lemmatization
df['tokens'] = df['tokens'].apply(lambda x: nltk.pos_tag(x))
print(df['tokens'][0])

[('horrific', 'NN'), ('alchemy', 'NN'), ('experiment', 'NN'), ('goes', 'VBZ'), ('wrong', 'JJ'), ('elric', 'JJ'), ('household', 'NN'), ('brothers', 'NNS'), ('edward', 'VBP'), ('alphonse', 'NN'), ('left', 'VBD'), ('catastrophic', 'JJ'), ('new', 'JJ'), ('reality', 'NN'), ('ignoring', 'VBG'), ('alchemical', 'JJ'), ('principle', 'NN'), ('banning', 'VBG'), ('human', 'JJ'), ('transmutation', 'NN'), ('boys', 'NNS'), ('attempted', 'VBD'), ('bring', 'VBG'), ('recently', 'RB'), ('deceased', 'VBN'), ('mother', 'NN'), ('back', 'RB'), ('life', 'NN'), ('instead', 'RB'), ('suffered', 'VBD'), ('brutal', 'JJ'), ('personal', 'JJ'), ('loss', 'NN'), ('alphonse', 'NN'), ('body', 'NN'), ('disintegrated', 'VBD'), ('edward', 'RB'), ('lost', 'VBN'), ('leg', 'NN'), ('sacrificed', 'VBN'), ('arm', 'JJ'), ('keep', 'NN'), ('alphonse', 'NN'), ('soul', 'NN'), ('physical', 'JJ'), ('realm', 'NN'), ('binding', 'VBG'), ('hulking', 'VBG'), ('suit', 'NN'), ('armor', 'NN'), ('brothers', 'NNS'), ('rescued', 'VBD'), ('neighbor

In [24]:
# convert tags to the tags that WordNetLemmatizer uses
def convert_pos(pos):
    if pos.startswith('V'):
        return wordnet.VERB
    elif pos.startswith('J'):
        return wordnet.ADJ
    elif pos.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# lemmatize words using the pos-tags
df['tokens'] = df['tokens'].apply(lambda x: [WordNetLemmatizer().lemmatize(w[0], pos=convert_pos(w[1])) for w in x])
print(df['tokens'][0])

['horrific', 'alchemy', 'experiment', 'go', 'wrong', 'elric', 'household', 'brother', 'edward', 'alphonse', 'leave', 'catastrophic', 'new', 'reality', 'ignore', 'alchemical', 'principle', 'ban', 'human', 'transmutation', 'boy', 'attempt', 'bring', 'recently', 'decease', 'mother', 'back', 'life', 'instead', 'suffer', 'brutal', 'personal', 'loss', 'alphonse', 'body', 'disintegrate', 'edward', 'lose', 'leg', 'sacrifice', 'arm', 'keep', 'alphonse', 'soul', 'physical', 'realm', 'bind', 'hulk', 'suit', 'armor', 'brother', 'rescue', 'neighbor', 'pinako', 'rockbell', 'granddaughter', 'winry', 'know', 'engineering', 'prodigy', 'winry', 'create', 'prosthetic', 'limb', 'edward', 'utilize', 'automail', 'tough', 'versatile', 'metal', 'use', 'robot', 'combat', 'armor', 'year', 'train', 'elric', 'brother', 'set', 'quest', 'restore', 'body', 'locate', 'philosopher', 'powerful', 'gem', 'allow', 'alchemist', 'defy', 'traditional', 'law', 'equivalent', 'exchange', 'edward', 'become', 'infamous', 'alchemi

In [25]:
# combine all tokens into one string
df['cleaned_string'] = df['tokens'].apply(lambda x: ' '.join(x))
print(df['cleaned_string'][0])

horrific alchemy experiment go wrong elric household brother edward alphonse leave catastrophic new reality ignore alchemical principle ban human transmutation boy attempt bring recently decease mother back life instead suffer brutal personal loss alphonse body disintegrate edward lose leg sacrifice arm keep alphonse soul physical realm bind hulk suit armor brother rescue neighbor pinako rockbell granddaughter winry know engineering prodigy winry create prosthetic limb edward utilize automail tough versatile metal use robot combat armor year train elric brother set quest restore body locate philosopher powerful gem allow alchemist defy traditional law equivalent exchange edward become infamous alchemist gain nickname fullmetal boy journey embroil grow conspiracy threatens fate world action adventure drama fantasy military shounen


In [26]:
df.columns

Index(['id', 'title', 'start_date', 'end_date', 'synopsis', 'mean', 'rank',
       'popularity', 'num_list_users', 'num_scoring_users', 'nsfw',
       'created_at', 'updated_at', 'media_type', 'status', 'num_episodes',
       'source', 'average_episode_duration', 'rating', 'medium_picture_url',
       'large_picture_url', 'alternative_title(s)', 'english_title',
       'japanese_title', 'year', 'season', 'genres', 'studios', 'summary',
       'tokens', 'cleaned_string'],
      dtype='object')

In [27]:
# combine title and english title
df['title'] = df['title'] + ' (' + df['english_title'] + ')'
df.head(3)

,id,title,start_date,end_date,synopsis,mean,rank,popularity,num_list_users,num_scoring_users,nsfw,created_at,updated_at,media_type,status,num_episodes,source,average_episode_duration,rating,medium_picture_url,large_picture_url,alternative_title(s),english_title,japanese_title,year,season,genres,studios,summary,tokens,cleaned_string
0,5114,Fullmetal Alchemist: Brotherhood (Fullmetal Al...,2009-04-05,2010-07-04,After a horrific alchemy experiment goes wrong...,9.1,1,3,3162045,2011391,white,2008-08-21T03:35:22+00:00,2023-04-02T18:07:03+00:00,tv,finished_airing,64,manga,1460,r,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,"Hagane no Renkinjutsushi: Fullmetal Alchemist,...",Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,2009,spring,"Action, Adventure, Drama, Fantasy, Military, S...",Bones,After a horrific alchemy experiment goes wrong...,"[horrific, alchemy, experiment, go, wrong, elr...",horrific alchemy experiment go wrong elric hou...
1,41467,Bleach: Sennen Kessen-hen (Bleach: Thousand-Ye...,2022-10-11,2022-12-27,Substitute Soul Reaper Ichigo Kurosaki spends ...,9.07,2,474,435441,207104,white,2020-03-18T09:10:15+00:00,2023-04-02T18:07:55+00:00,tv,finished_airing,13,manga,1471,r,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,Bleach: Thousand-Year Blood War Arc,Bleach: Thousand-Year Blood War,BLEACH 千年血戦篇,2022,fall,"Action, Adventure, Fantasy, Shounen",Pierrot,Substitute Soul Reaper Ichigo Kurosaki spends ...,"[substitute, soul, reaper, ichigo, kurosaki, s...",substitute soul reaper ichigo kurosaki spends ...
2,9253,Steins;Gate (Steins;Gate),2011-04-06,2011-09-14,Eccentric scientist Rintarou Okabe has a never...,9.07,3,13,2429238,1330574,white,2010-07-26T09:23:40+00:00,2023-04-02T18:08:42+00:00,tv,finished_airing,24,visual_novel,1460,pg_13,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,,Steins;Gate,STEINS;GATE,2011,spring,"Drama, Psychological, Sci-Fi, Suspense, Time, ...","White, Fox",Eccentric scientist Rintarou Okabe has a never...,"[eccentric, scientist, rintarou, okabe, thirst...",eccentric scientist rintarou okabe thirst scie...


In [28]:
# reorder and export the cleaned data
df = df[['id', 'title', 'alternative_title(s)', 'japanese_title', 'synopsis', 'genres', 'start_date', 'end_date', 'year', 'season', 
         'mean', 'rank', 'popularity', 'rating', 'nsfw', 'media_type', 'source', 'status', 'num_episodes', 'average_episode_duration', 
         'studios', 'created_at', 'updated_at', 'medium_picture_url', 'cleaned_string']]
# df.to_csv('data/cleaned_anime_data.csv', index=False)

# Generate Vectors using TF-IDF

In [29]:
cleaned_df = pd.read_csv('data/cleaned_anime_data.csv')
cleaned_df.head(3)

,id,title,alternative_title(s),japanese_title,synopsis,genres,start_date,end_date,year,season,mean,rank,popularity,rating,nsfw,media_type,source,status,num_episodes,average_episode_duration,studios,created_at,updated_at,medium_picture_url,cleaned_string
0,5114,Fullmetal Alchemist: Brotherhood (Fullmetal Al...,"Hagane no Renkinjutsushi: Fullmetal Alchemist,...",鋼の錬金術師 FULLMETAL ALCHEMIST,After a horrific alchemy experiment goes wrong...,"Action, Adventure, Drama, Fantasy, Military, S...",2009-04-05,2010-07-04,2009,spring,9.10,1,3,r,white,tv,manga,finished_airing,64,1460,Bones,2008-08-21T03:35:22+00:00,2023-04-02T18:07:03+00:00,https://api-cdn.myanimelist.net/images/anime/1...,horrific alchemy experiment go wrong elric hou...
1,41467,Bleach: Sennen Kessen-hen (Bleach: Thousand-Ye...,Bleach: Thousand-Year Blood War Arc,BLEACH 千年血戦篇,Substitute Soul Reaper Ichigo Kurosaki spends ...,"Action, Adventure, Fantasy, Shounen",2022-10-11,2022-12-27,2022,fall,9.07,2,474,r,white,tv,manga,finished_airing,13,1471,Pierrot,2020-03-18T09:10:15+00:00,2023-04-02T18:07:55+00:00,https://api-cdn.myanimelist.net/images/anime/1...,substitute soul reaper ichigo kurosaki spends ...
2,9253,Steins;Gate (Steins;Gate),NaN,STEINS;GATE,Eccentric scientist Rintarou Okabe has a never...,"Drama, Psychological, Sci-Fi, Suspense, Time, ...",2011-04-06,2011-09-14,2011,spring,9.07,3,13,pg_13,white,tv,visual_novel,finished_airing,24,1460,"White, Fox",2010-07-26T09:23:40+00:00,2023-04-02T18:08:42+00:00,https://api-cdn.myanimelist.net/images/anime/1...,eccentric scientist rintarou okabe thirst scie...


In [48]:
# transform string to TD-IDF vector and ignore terms that have a document frequency higher than 50% and lower than 1%
tf_vec = TfidfVectorizer(ngram_range=(1,2), max_df=0.5, min_df=0.01)
tfidf_matrix = tf_vec.fit_transform(cleaned_df['cleaned_string'].astype('str'))
# print(tf_vec.get_feature_names_out())

In [49]:
tfidf_array = tfidf_matrix.toarray()

In [50]:
# calculate cos similarity
cos_sim = cosine_similarity(tfidf_array, tfidf_array)
print(cos_sim, '\n')
print(cos_sim.shape)

[[1.         0.16329505 0.06506319 ... 0.02602815 0.08942505 0.06261767]
 [0.16329505 1.         0.034972   ... 0.07107613 0.02963231 0.03737152]
 [0.06506319 0.034972   1.         ... 0.06373594 0.13779335 0.05199522]
 ...
 [0.02602815 0.07107613 0.06373594 ... 1.         0.00851976 0.04983595]
 [0.08942505 0.02963231 0.13779335 ... 0.00851976 1.         0.05122277]
 [0.06261767 0.03737152 0.05199522 ... 0.04983595 0.05122277 1.        ]] 

(6836, 6836)


# Build Recommendation Engine Based on Similarity Score

In [51]:
# create indices for animes
indices = pd.Series(cleaned_df.index, index=cleaned_df['title'])
print(indices)

title
Fullmetal Alchemist: Brotherhood (Fullmetal Alchemist: Brotherhood)                                                             0
Bleach: Sennen Kessen-hen (Bleach: Thousand-Year Blood War)                                                                     1
Steins;Gate (Steins;Gate)                                                                                                       2
Shingeki no Kyojin Season 3 Part 2 (Attack on Titan Season 3 Part 2)                                                            3
Shingeki no Kyojin: The Final Season - Kanketsu-hen (Attack on Titan: The Final Season - Final Chapters)                        4
                                                                                                                             ... 
Code Geass: Hangyaku no Lelouch R2 Omake Flash (Code Geass: Lelouch of the Rebellion R2 Extra Flash - Baba Theater Redux)    6831
Diabolik Lovers OVA (<NA>)                                                          

In [52]:
# recommendation engine based on cosine similarity
def get_recommendations(title, cosine_sim, indices):
    # get index of the title requested
    idx = indices[title]
    # use index to sort the row in the cosine similarity matrix and get top 10 indices
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    first = 1
    last = 21
    top_20 = sim_scores[first:last]
    anime_indices = [i[0] for i in top_20]
    animes_scores = [i[1] for i in top_20]
    
    # return the top 10 most similar animes that doesn't contain the title
    results = cleaned_df[['title', 'alternative_title(s)', 'synopsis', 'genres', 'studios', 'mean', 'year', 'status', 'medium_picture_url']].iloc[anime_indices]
    results['cosine_similarity_score'] = animes_scores
    # filter out animes that have a mean (rating) lower than 5
    filtered_results = results[results['mean']>=5]
    # check if results contain the words (longer than 5 letters) in the title, if yes, remove the anime
    title = re.sub('[()]', '', str(title))
    title = list(set((title.split())))
    title_word_list = [x for x in title if len(x)>5]
    if not title_word_list:
        pass
    else:
        for x in title_word_list:
            filtered_results = filtered_results[filtered_results['title'].str.contains(x)==False]
    
    # check how many rows removed/filtered out, if none then just return the results. If yes, then add new recommendation until no sequels exists in the top 10
    diff = results.shape[0] - filtered_results.shape[0]
    
    if diff == 0:
        return filtered_results
    else:
        add_recommendations = pd.DataFrame(columns=['title', 'alternative_title(s)', 'synopsis', 'genres', 'studios', 'mean', 'year', 'status', 'medium_picture_url', 'cosine_similarity_score'])
        while diff > 0:
            new_idx = sim_scores[last:last+diff]
            last = last + diff
            new_anime_idx = [i[0] for i in new_idx]
            new_animes_scores = [i[1] for i in new_idx]
            new_recommendations = cleaned_df[['title', 'alternative_title(s)', 'synopsis', 'genres', 'studios', 'mean', 'year', 'status', 'medium_picture_url']].iloc[new_anime_idx]
            new_recommendations['cosine_similarity_score'] = new_animes_scores
            filtered_new_recommendations = new_recommendations[new_recommendations['mean']>=5]
            for i in title_word_list:
                filtered_new_recommendations = filtered_new_recommendations[filtered_new_recommendations['title'].str.contains(i)==False]

            add_recommendations = pd.concat([add_recommendations, filtered_new_recommendations])    
            diff = new_recommendations.shape[0] - filtered_new_recommendations.shape[0]

        new_results = pd.concat([filtered_results, add_recommendations])
        return new_results

In [53]:
# example
recommendations = get_recommendations('Oshi no Ko (Oshi No Ko)', cos_sim, indices)
recommendations

,title,alternative_title(s),synopsis,genres,studios,mean,year,status,medium_picture_url,cosine_similarity_score
1414,PriPara (<NA>),"Puri Para, Pri Para, PuriPara, Prism Paradise",Every little girl waits for the day she'll get...,"Idols, (Female), Music, School, Shoujo, Slice,...","Tatsunoko, Production, Dongwoo, A&E",7.50,2014,finished_airing,https://api-cdn.myanimelist.net/images/anime/2...,0.344757
6625,Love Live! Sunshine!! Photo Session (<NA>),Love Live! Sunshine!! The School Idol Movie: O...,Short theater manners shown before Love Live! ...,"Comedy, Idols, (Female)",Sunrise,5.80,2019,finished_airing,https://api-cdn.myanimelist.net/images/anime/1...,0.325429
1267,Shine Post (<NA>),NaN,"Haru Nabatame, Rio Seibu, and Kyouka Tamaki's ...","Idols, (Female), Music, Slice, of, Life, Super...","Studio, Kai",7.55,2022,finished_airing,https://api-cdn.myanimelist.net/images/anime/1...,0.323810
1486,Kiratto Pri☆chan Season 2 (<NA>),Kiratto Pri☆chan 2nd Season,"Mirai Momoyama, Emo Moegi, and Rinka Aoba are ...","Comedy, Idols, (Female), Music, Shoujo, Slice,...","Tatsunoko, Production",7.47,2019,finished_airing,https://api-cdn.myanimelist.net/images/anime/1...,0.321692
3855,Kami Kuzu☆Idol (Phantom of the Idol),Kami Kuzu Idol,Kazuki Yoshino and Yuuya Niyodo are idols who ...,"Comedy, Idols, (Female), Idols, (Male), Josei,...","Studio, Gokumi",6.74,2022,finished_airing,https://api-cdn.myanimelist.net/images/anime/1...,0.313566
5490,Sekkou Boys (Sekko Boys),NaN,"As luck would have it, on her first day at Hol...","Anthropomorphic, Comedy, Idols, (Male), Music",LIDENFILMS,6.25,2016,finished_airing,https://api-cdn.myanimelist.net/images/anime/5...,0.312988
1942,Idol Time PriPara (<NA>),NaN,"The story focuses on Yui, a girl who lives in ...","Idols, (Female), Music, School, Shoujo","Tatsunoko, Production, Dongwoo, A&E",7.33,2017,finished_airing,https://api-cdn.myanimelist.net/images/anime/3...,0.312330
5223,Love Live! The School Idol Movie: Gekijou Mann...,Love Live! Theater Manner Commercial Movie,Short videos shown in theaters before Love Liv...,"Comedy, Idols, (Female)",Sunrise,6.33,2015,finished_airing,https://api-cdn.myanimelist.net/images/anime/1...,0.307738
4563,"Wake Up, Girls! Shin Shou (Wake Up, Girls! New...","Wake Up, Girls! Shin Sho","""Wake Up, Girls!"" is the story of growth and f...","Drama, Idols, (Female), Music",Millepensee,6.53,2017,finished_airing,https://api-cdn.myanimelist.net/images/anime/1...,0.304421
2992,UniteUp! (<NA>),NaN,"Akira Kiyose, a high schooler who loves to sin...","Idols, (Male), Music, Slice, of, Life",CloverWorks,7.02,2023,finished_airing,https://api-cdn.myanimelist.net/images/anime/1...,0.293164


In [55]:
# sort results by year
recommendations.sort_values(by='year', ascending=False)

,title,alternative_title(s),synopsis,genres,studios,mean,year,status,medium_picture_url,cosine_similarity_score
1124,The IDOLM@STER Cinderella Girls: U149 (<NA>),NaN,It's time to pull back the curtain on a new Ci...,"Idols, (Female), Music",CygamesPictures,7.61,2023,currently_airing,https://api-cdn.myanimelist.net/images/anime/1...,0.259580
2992,UniteUp! (<NA>),NaN,"Akira Kiyose, a high schooler who loves to sin...","Idols, (Male), Music, Slice, of, Life",CloverWorks,7.02,2023,finished_airing,https://api-cdn.myanimelist.net/images/anime/1...,0.293164
942,Love Live! Nijigasaki Gakuen School Idol Douko...,Love Live! Nijigasaki High School Idol Club 2n...,The Nijigasaki High School Idol Club continues...,"Idols, (Female), Music, School, Slice, of, Life",Sunrise,7.70,2022,finished_airing,https://api-cdn.myanimelist.net/images/anime/1...,0.292416
1267,Shine Post (<NA>),NaN,"Haru Nabatame, Rio Seibu, and Kyouka Tamaki's ...","Idols, (Female), Music, Slice, of, Life, Super...","Studio, Kai",7.55,2022,finished_airing,https://api-cdn.myanimelist.net/images/anime/1...,0.323810
3855,Kami Kuzu☆Idol (Phantom of the Idol),Kami Kuzu Idol,Kazuki Yoshino and Yuuya Niyodo are idols who ...,"Comedy, Idols, (Female), Idols, (Male), Josei,...","Studio, Gokumi",6.74,2022,finished_airing,https://api-cdn.myanimelist.net/images/anime/1...,0.313566
4156,Eternity Memories (Eternity Memories),THE IDOLM@STER CINDERELLA GIRLS 10th Anniversa...,An animation commemorating the 10th anniversar...,"Fantasy, Idols, (Female)",CygamesPictures,6.65,2022,finished_airing,https://api-cdn.myanimelist.net/images/anime/1...,0.259585
5077,Tokyo 7th Sisters: Bokura wa Aozora ni Naru (<...,T7S,"In the year 2032, the legendary idol unit ""7th...","Comedy, Idols, (Female), Music, School, Slice,...","LandQ, studios",6.38,2021,finished_airing,https://api-cdn.myanimelist.net/images/anime/1...,0.282702
1372,Voy@ger (<NA>),The iDOLM@STER Series Concept Movie 2021,Voy@ger is conceptual movie produced in commem...,"Idols, (Female), Idols, (Male), Music, Sci-Fi","Khara, CloverWorks",7.52,2021,finished_airing,https://api-cdn.myanimelist.net/images/anime/1...,0.259078
1654,Idoly Pride (Idoly Pride),NaN,The VENUS Program is an idol scoring system th...,"Idols, (Female), Music",Lerche,7.42,2021,finished_airing,https://api-cdn.myanimelist.net/images/anime/1...,0.280057
1486,Kiratto Pri☆chan Season 2 (<NA>),Kiratto Pri☆chan 2nd Season,"Mirai Momoyama, Emo Moegi, and Rinka Aoba are ...","Comedy, Idols, (Female), Music, Shoujo, Slice,...","Tatsunoko, Production",7.47,2019,finished_airing,https://api-cdn.myanimelist.net/images/anime/1...,0.321692
